<h1> Creating TensorFlow model </h1>

This notebook illustrates:
<ol>
<li> Creating a model using the high-level Estimator API 
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'ryan-asl'
PROJECT = 'billingtest-175516'
REGION = 'us-central1'

In [2]:
# Import os environment variables
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

Make bucket if bucket doesn't exist

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

Check to make sure the training and evaluation CSVs are in the correct location

In [3]:
%bash
ls *.csv

eval.csv
train.csv


<h2> Create TensorFlow model using TensorFlow's Estimator API </h2>
<p>
First, write an input_fn to read the data.

In [4]:
# Import modules
import shutil
import numpy as np
import tensorflow as tf

In [5]:
# Determine CSV, label, and key columns
CSV_COLUMNS = 'weight_pounds,is_male,mother_age,mother_race,plurality,gestation_weeks,mother_married,cigarette_use,alcohol_use,key'.split(',')
LABEL_COLUMN = 'weight_pounds'
KEY_COLUMN = 'key'

# Set default values for each CSV column
DEFAULTS = [[0.0], ['null'], [0.0], ['null'], [0.0], [0.0], ['null'], ['null'], ['null'], ['nokey']]

In [6]:
# Define some hyperparameters
TRAIN_STEPS = 1000
BUFFER_SIZE = 512
BATCH_SIZE = 32
TRAIN_REPEAT_COUNT = 8
EVAL_REPEAT_COUNT = 1
EMBED_SIZE = 3
HIDDEN_UNITS = [64, 16, 4]

In [17]:
# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API
def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults=DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label
    
    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)

    # Create dataset from file list
    dataset = (tf.data.TextLineDataset(file_list)  # Read text file
                 .map(decode_csv))  # Transform each elem by applying decode_csv fn
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this
 
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn

Next, define the feature columns

In [18]:
# Define feature columns
def get_wide_deep():
  # Define column types
  races = ['White', 'Black', 'American Indian', 'Chinese', 
           'Japanese', 'Hawaiian', 'Filipino', 'Unknown',
           'Asian Indian', 'Korean', 'Samaon', 'Vietnamese']
  is_male, mother_age, mother_race, plurality, gestation_weeks, mother_married, cigarette_use, alcohol_use = \
   [ \
    tf.feature_column.categorical_column_with_vocabulary_list('is_male', 
                        ['True', 'False', 'Unknown']),
    tf.feature_column.numeric_column('mother_age'),
    tf.feature_column.categorical_column_with_vocabulary_list('mother_race', 
                        races),
    tf.feature_column.numeric_column('plurality'),
    tf.feature_column.numeric_column('gestation_weeks'),
    tf.feature_column.categorical_column_with_vocabulary_list('mother_married', 
                        ['True', 'False']),
    tf.feature_column.categorical_column_with_vocabulary_list('cigarette_use', 
                        ['True', 'False', 'None']),
    tf.feature_column.categorical_column_with_vocabulary_list('alcohol_use', 
                        ['True', 'False', 'None'])
    ]

  # Discretize/bucketize continuous features
  mother_age_buckets = tf.feature_column.bucketized_column(mother_age, 
                        boundaries=np.arange(15, 45, 1).tolist())
  plurality_buckets = tf.feature_column.bucketized_column(plurality, 
                        boundaries=np.arange(0.5, 5.5, 1.0).tolist())
  gestation_buckets = tf.feature_column.bucketized_column(gestation_weeks, 
                        boundaries=np.arange(17, 47, 1).tolist())
  
  # Feature cross all three bucketized feature columns to create a sparse representation
  buckets_crossed = tf.feature_column.crossed_column([mother_age_buckets, 
                                                      plurality_buckets, 
                                                      gestation_buckets], 
                                                     hash_bucket_size = 20000)
  
  # Embed mother race into a lower dimensional continuous space
  mother_race_embed = tf.feature_column.embedding_column(mother_race, EMBED_SIZE)

  # Sparse columns are wide, have a linear relationship with the output
  wide = [is_male, mother_race, 
          mother_age_buckets, plurality_buckets, gestation_buckets,
          mother_married, cigarette_use, alcohol_use, buckets_crossed]
  
  # Cross all wide columns and embed into a lower dimension
  wide_crossed = tf.feature_column.crossed_column(wide, hash_bucket_size = 20000)
  wide_embed = tf.feature_column.embedding_column(wide_crossed, EMBED_SIZE)

  # Continuous columns are deep, have a complex relationship with the output
  deep = [mother_age,
          mother_race_embed,
          plurality,
          gestation_weeks,
          wide_embed
               ]
  return wide, deep

To predict with the TensorFlow model, we also need a serving input function. We will want all the inputs from our user.

In [19]:
# Create serving input function to be able to serve predictions later using provided inputs
def serving_input_fn():
  feature_placeholders = {
    'is_male': tf.placeholder(tf.string, [None]),
    'mother_age': tf.placeholder(tf.float32, [None]),
    'mother_race': tf.placeholder(tf.string, [None]),
    'plurality': tf.placeholder(tf.float32, [None]),
    'gestation_weeks': tf.placeholder(tf.float32, [None]),
    'mother_married': tf.placeholder(tf.string, [None]),
    'cigarette_use': tf.placeholder(tf.string, [None]),
    'alcohol_use': tf.placeholder(tf.string, [None])
  }
  features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [20]:
# Create estimator to train and evaluate
def train_and_evaluate(output_dir):
  wide, deep = get_wide_deep()
  estimator = tf.estimator.DNNLinearCombinedRegressor(
                       model_dir = output_dir,
                       linear_feature_columns = wide,
                       dnn_feature_columns = deep,
                       dnn_hidden_units = HIDDEN_UNITS)
  train_spec = tf.estimator.TrainSpec(
                       input_fn = read_dataset('train.csv', mode = tf.estimator.ModeKeys.TRAIN),
                       max_steps = TRAIN_STEPS)
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec = tf.estimator.EvalSpec(
                       input_fn = read_dataset('eval.csv', mode = tf.estimator.ModeKeys.EVAL),
                       steps = None,
                       start_delay_secs = 60, # start evaluating after N seconds
                       throttle_secs = 300,  # evaluate every N seconds
                       exporters = exporter)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Finally, train!

In [21]:
# Run the model
shutil.rmtree('babyweight_trained', ignore_errors = True) # start fresh each time
train_and_evaluate('babyweight_trained')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efd37518fd0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'babyweight_trained', '_save_summary_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into babyweight_trained/model.ckpt.
INFO:tensorflow:loss = 44982.938, step = 1
INFO:tensorflow:global_step/sec: 8.60145
I

When I ran it, the final lines of the output (above) were:
<pre>
INFO:tensorflow:Saving dict for global step 1000: average_loss = 1.1695355, global_step = 1000, loss = 37.41367
INFO:tensorflow:Performing the final export in the end of training.
INFO:tensorflow:Restoring parameters from babyweight_trained/model.ckpt-1000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: babyweight_trained/export/exporter/temp-1517330487/saved_model.pb
</pre>
The exporter directory contains the final model and the final RMSE (the average_loss) is 1.1695355

<h2> Monitor and experiment with training </h2>

In [ ]:
from google.datalab.ml import TensorBoard
my_pid = TensorBoard().start('./babyweight_trained')

In TensorBoard, look at the learned embeddings for the race. Are they getting clustered? How about the weights for the hidden layers? What if you run this longer? What happens if you change the batchsize?

In [ ]:
TensorBoard.stop(my_pid)

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License